In [1]:
from networks.UNet import LitUNet
from lightning import Trainer
from torch.nn.functional import softmax
import torchvision
from transforms import Resize

In [2]:
import os
from skimage.io import imread
from skimage.transform import rescale, resize
import numpy as np
import matplotlib.pyplot as plt
import torch
from glob import glob

In [3]:
from data.dataset import OEMMiniDataset
from torch.utils.data import DataLoader

In [4]:
map_location = {'cuda:0':'cpu'}
ckpt_pth = glob('/home/ubuntu/hrl/oem_mini_experiments/.neptune/Untitled/MIN-38/checkpoints/*.ckpt')
model = LitUNet.load_from_checkpoint(checkpoint_path=ckpt_pth[0], map_location='cpu')
trainer = Trainer(accelerator='cpu',num_nodes=1)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/pytorch/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/conda/envs/pytorch/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [5]:
import numpy as np
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral, create_pairwise_gaussian
d = dcrf.DenseCRF2D(512, 512, 9)  # width, height, nlabels

In [6]:
from matplotlib.pyplot import figure

def pred_labels(trainer, dataloader, im):
    preds = trainer.predict(model, dataloaders=dataloader)
    print(preds[0][0].shape)
    print(preds[0][1].shape)
    print(preds[0][2].shape)
    pred_soft_flatten = preds[0][2].squeeze().transpose(2,0,1).reshape(9,-1)
    
    U = unary_from_softmax(pred_soft_flatten)

    # U = -np.log(U)
    d.setUnaryEnergy(U)

    d.addPairwiseBilateral(sxy=(9,9), srgb=(256,256,256), rgbim=im, compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)
    # d.addPairwiseGaussian(sxy=3, compat=3)
    Q = d.inference(3)
    pred = np.argmax(Q, axis=0).reshape((512, 512))
    return pred, preds[0][0]
    # return preds[0][0]



def viz_data(image_pth):
    img = imread(image_pth)
    img_list = glob(image_pth)
    val_augm = torchvision.transforms.Compose(
                    [
                        Resize(512),
                    ],
                )
    predict_loader = DataLoader(OEMMiniDataset(img_list, augm=val_augm), batch_size=1)
    crf, preds = pred_labels(trainer=trainer, dataloader=predict_loader, im=resize(img, (512, 512), anti_aliasing=True))
    plt.rcParams["figure.figsize"] = (16,4)
    try:
        label = imread(image_pth.replace('/images/', '/labels/'))
        pl, pc= np.unique(preds, return_counts=True)
        al, ac= np.unique(label, return_counts=True)
        f, imx = plt.subplots(1,4)
        imx[0].imshow(img)
        imx[0].set_title('RGB')
        imx[1].imshow(label)
        imx[1].set_title('GT')
        imx[2].imshow(preds)
        imx[2].set_title('Predicted')
        imx[3].imshow(crf)
        imx[3].set_title('CRF refined')
        print(f'Pred Labels found: {pl}')
        np.set_printoptions(precision=3, suppress=True)
        print(f'Pred percentages: {pc/np.sum(pc)*100}')
        print(f'Actual Labels found: {al}')
        np.set_printoptions(precision=3, suppress=True)
        print(f'Pred percentages: {ac/np.sum(ac)*100}')
        
    except:
        pl, pc= np.unique(preds, return_counts=True)
        f, imx = plt.subplots(1,2)
        imx[0].imshow(img)
        imx[1].imshow(preds)
        print(f'Pred Labels found: {pl}')
        np.set_printoptions(precision=3, suppress=True)
        print(f'Label percentages: {pc/np.sum(pc)*100}')

In [7]:
pred_img_path = sorted(glob('/home/ubuntu/hrl/oem_mini_experiments/data/processing/OpenEarthMap_Mini/*/images/*.tif'))[25]
viz_data(pred_img_path)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

(512, 512, 1)
(512, 512, 1)
(512, 512, 9, 1)


ValueError: Buffer dtype mismatch, expected 'unsigned char' but got 'double'